In [761]:
full = False
input_file = 'input_full.txt' if full else 'input_example.txt'

with open(input_file) as file:
    lines = file.readlines()
lines = [line for line in lines]

# Detect map
map = []
for i in range(len(lines)-2):
    map += [lines[i].replace('\n', '')]
    
# Detect directions:
directions = lines[len(lines)-1]
# - Number of moves:
nbrs = [int(x) for x in directions.replace('L', 'R').split('R')]
# - Turns:
turns = list(''.join(i for i in directions if not i.isdigit()))

In [762]:
# Load the map
points = {}

def get_min_colid(rowid):
    return min([c for (r,c) in points.keys() if r == rowid])

def get_min_rowid(colid):
    return min([r for (r,c) in points.keys() if c == colid])

class pt:
    def __init__(self, rowid, colid, type):
        self.rowid = rowid # Row number
        self.colid = colid # Col number
        self.type = type   # Can be '#' or '.'
        self.fnum = -1     # face number

# Parse all rows
for i in range(len(map)):
    rowid = i + 1
    # Parse all columns
    for j in range(len(map[i])):
        colid = j + 1
        char      = map[i][j]
        char_prev = map[i][j-1] if (colid > 1) else None
        char_next = map[i][j+1] if (colid < len(map[i])) else None
        char_up   = map[i-1][j] if (rowid > 1) and colid <= len(map[i-1]) else None
        char_down = map[i+1][j] if (rowid < len(map)) and colid <= len(map[i+1]) else None
        
        if char != ' ':
            p = pt(rowid, colid, char)
            points[(rowid, colid)] = p
            if (char_prev is not None) and (char_prev != ' '):
                p.left = points[(rowid, colid-1)]
                p.left.right = p               
            if (char_next is None) or (char_next == ' '):
                min_colid = get_min_colid(rowid)
                p.right = points[(rowid, min_colid)]
                p.right.left = p
            if (char_up is not None) and (char_up != ' '):
                p.up = points[(rowid-1, colid)]
                p.up.down = p
            if (char_down is None) or (char_down == ' '):
                min_rowid = get_min_rowid(colid)
                p.down = points[(min_rowid, colid)]
                p.down.up = p

In [763]:
# Part 1
facing = 0 # First direction is to the right
p = points[(1, get_min_colid(1))] # First point is the first on the top left

for step in range(len(nbrs)):
    # Move until nb of steps or until # is found
    for i in range(nbrs[step]):
        if facing == 0:
            p_next = p.right
        elif facing == 1:
            p_next = p.down
        elif facing == 2:
            p_next = p.left
        elif facing == 3:
            p_next = p.up
        
        if p_next.type == '#':
                break
        else:
            p = p_next
    
    if step == len(nbrs)-1:
        break
    
    turn = turns[step]
    if turn == 'L':
        facing = (facing - 1) % 4
    else:
        facing = (facing + 1) % 4

print(f'Part 1: {1000 * p.rowid + 4 * p.colid + facing}')

Part 1: 6032


In [764]:
# Part 2
points2 = points.copy()

# Face size
a = 50 if full else 4

def l(start, end=None):
    if end is None:
        return [start] * a
    elif start < end:
        return list(range(0,1000))[start:end+1]
    else:
        return (list(range(0,1000))[end:start+1])[::-1]

def join_faces(fa, fb):
    fnuma, fnumb = fa[0], fb[0]
    fa_dir, fb_dir = fa[1], fb[1]

    # Define direction after going from face fa => fb
    if fa_dir == 'right':
        facing_changes[(fnumb, fnuma)] = 2
    elif fa_dir == 'down':
        facing_changes[(fnumb, fnuma)] = 3
    elif fa_dir == 'left':
        facing_changes[(fnumb, fnuma)] = 0
    elif fa_dir == 'up':
        facing_changes[(fnumb, fnuma)] = 1

    # Define direction after going from face fb => fa
    if fb_dir == 'right':
        facing_changes[(fnuma, fnumb)] = 2
    elif fb_dir == 'down':
        facing_changes[(fnuma, fnumb)] = 3
    elif fb_dir == 'left':
        facing_changes[(fnuma, fnumb)] = 0
    elif fb_dir == 'up':
        facing_changes[(fnuma, fnumb)] = 1

    # Redefine edges
    for rowida, colida, rowidb, colidb in zip(fa[2], fa[3], fb[2], fb[3]):
        pa, pb = points2[(rowida, colida)], points2[(rowidb, colidb)]
        pa.fnum, pb.fnum = fnuma, fnumb

        if fa_dir == 'up':
            pa.up = pb
        elif fa_dir == 'right':
            pa.right = pb
        elif fa_dir == 'down':
            pa.down = pb
        elif fa_dir == 'left':
            pa.left = pb
        
        if fb_dir == 'up':
            pb.up = pa
        elif fb_dir == 'right':
            pb.right = pa
        elif fb_dir == 'down':
            pb.down = pa
        elif fb_dir == 'left':
            pb.left = pa

facing_changes = {}

# Define the connections between faces/edges
if full:
    # 1 up      6 left
    f1_up   = 1, 'up',   l(1),          l(a+1, 2*a)
    f6_left = 6, 'left', l(3*a+1, 4*a), l(1)
    join_faces(f1_up, f6_left)

    # 1 left    4 left
    f1_left = 1, 'left', l(1,a),        l(a+1)
    f4_left = 4, 'left', l(3*a, 2*a+1), l(1)
    join_faces(f1_left, f4_left)

    # 2 up      6 down
    f2_up   = 2, 'up',   l(1),   l(2*a+1, 3*a)
    f6_down = 6, 'down', l(4*a), l(1, a)
    join_faces(f2_up, f6_down)

    # 2 right   5 right
    f2_right = 2, 'right', l(1, a),       l(3*a)
    f5_right = 5, 'right', l(3*a, 2*a+1), l(2*a)
    join_faces(f2_right, f5_right)

    # 2 down    3 right
    f2_down  = 2, 'down',  l(a), l(2*a+1, 3*a)
    f3_right = 3, 'right', l(a+1, 2*a), l(2*a)
    join_faces(f2_down, f3_right)

    # 3 left    4 up
    f3_left = 3, 'left', l(a+1, 2*a), l(a+1)
    f4_up   = 4, 'up',   l(2*a+1),    l(1, a)
    join_faces(f3_left, f4_up)

     # 5 down    6 right
    f5_down  = 5, 'down',  l(3*a), l(a+1, 2*a)
    f6_right = 6, 'right', l(3*a+1, 4*a), l(a)
    join_faces(f5_down, f6_right)

else:
    # 1 up      2 up
    f1_up = 1, 'up', l(1),   l(2*a+1, 3*a)
    f2_up = 2, 'up', l(a+1), l(a,1)
    join_faces(f1_up, f2_up)

    # 1 right   6 right
    f1_right = 1, 'right', l(1,a), l(3*a)
    f6_right = 6, 'right', l(3*a,  2*a+1), l(4*a)
    join_faces(f1_right, f6_right)

    # 1 left    3 up
    f1_left = 1, 'left', l(1,a), l(2*a+1)
    f3_up   = 3, 'up',   l(a+1), l(a+1, 2*a)
    join_faces(f1_left, f3_up)

    # 2 down    5 down
    f2_down = 2, 'down', l(2*a), l(1,a)
    f5_down = 5, 'down', l(3*a), l(3*a, 2*a+1)
    join_faces(f2_down, f5_down)

    # 2 left    6 down
    f2_left = 2, 'left', l(4+1,2*a), l(1)
    f6_down = 6, 'down', l(3*a),     l(4*a, 3*a+1)
    join_faces(f2_left, f6_down)

    # 3 down    5 left
    f3_down = 3, 'down', l(2*a), l(4+1, 2*a)
    f5_left = 5, 'left', l(3*a,  2*a+1), l(2*a+1)
    join_faces(f3_down, f5_left)

    # 4 right   6 up
    f4_right = 4, 'right', l(a+1, 2*a), l(3*a)
    f6_up    = 6, 'up',    l(2*a+1),    l(4*a, 3*a+1)
    join_faces(f4_right, f6_up)


In [765]:
facing = 0 # First direction is to the right
p = points2[(1, get_min_colid(1))] # First point is the first on the top left

for step in range(len(nbrs)):
    # Move until nb of steps or until # is found
    for i in range(nbrs[step]):
        if facing == 0:
            p_next = p.right
        elif facing == 1:
            p_next = p.down
        elif facing == 2:
            p_next = p.left
        elif facing == 3:
            p_next = p.up
        
        if p_next.type == '#':
                break
        else:
            p_prev = p
            p = p_next
            # if we go from one face to another, we need to update the direction
            fnump_prev, fnump_next = p_prev.fnum, p_next.fnum
            if (fnump_prev != -1) and (fnump_next != -1) and (fnump_prev != fnump_next):
                if (fnump_prev, fnump_next) in facing_changes:
                    facing = facing_changes[(fnump_prev, fnump_next)]
            
    if step == len(nbrs)-1:
        break
    
    turn = turns[step]
    if turn == 'L':
        facing = (facing - 1) % 4
    else:
        facing = (facing + 1) % 4

print(f'Part 2: {1000 * p.rowid + 4 * p.colid + facing}')

Part 2: 5031


In [766]:
# Example:
#     11
#     11
# 223344
# 223344
#     5566
#     5566

# 1 up      2 up
# 1 right   6 right
# 1 down    4 up     OK
# 1 left    3 up
#(2 up      1 up)
# 2 right   3 left   OK
# 2 down    5 down
# 2 left    6 down
#(3 up      1 left)
# 3 right   4 left   OK
# 3 down    5 left
#(3 left    2 right)
#(4 up      1 down)
# 4 right   6 up
# 4 down    5 up     OK
#(4 left    3 right)
#(5 up      4 down)
# 5 right   6 left   OK
#(5 down    2 down)
#(5 left    3 down)
#(6 up      4 right)
#(6 right   1 right)
#(6 down    2 left)
#(6 left    5 right)


# Full:
#   1122
#   1122
#   33
#   33
# 4455
# 4455
# 66
# 66

# 1 up      6 left
# 1 right   2 left    OK
# 1 down    3 up      OK
# 1 left    4 left
# 2 up      6 down
# 2 right   5 right
# 2 down    3 right
#(2 left    1 right)
#(3 up      1 down)
#(3 right   2 down)
# 3 down    5 up      OK
# 3 left    4 up
#(4 up      3 left)
# 4 right   5 left    OK
# 4 down    6 up      OK
#(4 left    1 left)
#(5 up      3 down)
#(5 right   2 right)
# 5 down    6 right
#(5 left    4 right)
#(6 up      4 down)
#(6 right   5 down)
#(6 down    2 up)
#(6 left    1 up)

